# MI

In [3]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

# ---- SETTINGS ----
channels = ['C3_ref', 'CZ_ref', 'C4_ref']
samples_per_trial = 1750
n_trials_per_session = 10
test_subjects = [f'S{i}' for i in range(36, 41)]  # S36 to S40
data_path = r"C:\Users\zezom\PycharmProjects\HorusEye\Data\Test\MI"  # where preprocessed csvs are
model_path = r"C:\Users\zezom\PycharmProjects\HorusEye\best_mi_cnn_model.keras"  # trained model
output_csv = r"C:\Users\zezom\PycharmProjects\HorusEye\mi_test_predictions.csv"

# Load label encoder classes
mi_labels = ['Left', 'Right']

# Load model
model = tf.keras.models.load_model(model_path)
print("[INFO] Model loaded.")

def load_trial(subject, session, trial, data_path, channels, samples_per_trial):
    file = f"{subject}_{session}_EEGdata_preprocessed.csv"
    path = os.path.join(data_path, file)
    if not os.path.exists(path):
        print(f"[WARN] Missing file: {path}")
        return None
    df = pd.read_csv(path)
    start = (trial - 1) * samples_per_trial
    end = trial * samples_per_trial
    # Check bounds
    if end > len(df):
        print(f"[WARN] Trial {trial} out of bounds in {file}")
        return None
    trial_data = df[channels].iloc[start:end].values.T
    if trial_data.shape[1] != samples_per_trial:
        print(f"[WARN] Bad shape for {file} trial {trial}: {trial_data.shape}")
        return None
    # Normalize (z-score per trial, per channel)
    trial_data = (trial_data - trial_data.mean(axis=-1, keepdims=True)) / (trial_data.std(axis=-1, keepdims=True) + 1e-6)
    return trial_data

# --- Predict for S36 to S40, sessions 1, trials 1–10 ---
results = []

for subject in test_subjects:
    session = 1  # you may need to loop over sessions if your data has multiple
    for trial in range(1, n_trials_per_session + 1):
        X_trial = load_trial(subject, session, trial, data_path, channels, samples_per_trial)
        if X_trial is None:
            pred_label = "Missing"
        else:
            X_trial = np.expand_dims(X_trial, axis=0)  # shape: (1, n_channels, samples_per_trial)
            pred_prob = model.predict(X_trial, verbose=0)
            pred_class = np.argmax(pred_prob, axis=1)[0]
            pred_label = mi_labels[pred_class]
        results.append({
            "subject": subject,
            "session": session,
            "trial": trial,
            "prediction": pred_label
        })
        print(f"[{subject}][Trial {trial}] --> {pred_label}")

# --- Save Results to CSV ---
results_df = pd.DataFrame(results)
results_df.to_csv(output_csv, index=False)
print(f"\nAll predictions saved to {output_csv}")


[INFO] Model loaded.
[S36][Trial 1] --> Left
[S36][Trial 2] --> Left
[S36][Trial 3] --> Left
[S36][Trial 4] --> Left
[S36][Trial 5] --> Left
[S36][Trial 6] --> Right
[S36][Trial 7] --> Left
[S36][Trial 8] --> Left
[S36][Trial 9] --> Left
[S36][Trial 10] --> Left
[S37][Trial 1] --> Left
[S37][Trial 2] --> Left
[S37][Trial 3] --> Right
[S37][Trial 4] --> Left
[S37][Trial 5] --> Left
[S37][Trial 6] --> Left
[S37][Trial 7] --> Left
[S37][Trial 8] --> Left
[S37][Trial 9] --> Left
[S37][Trial 10] --> Left
[S38][Trial 1] --> Left
[S38][Trial 2] --> Right
[S38][Trial 3] --> Left
[S38][Trial 4] --> Left
[S38][Trial 5] --> Left
[S38][Trial 6] --> Left
[S38][Trial 7] --> Left
[S38][Trial 8] --> Left
[S38][Trial 9] --> Left
[S38][Trial 10] --> Left
[S39][Trial 1] --> Left
[S39][Trial 2] --> Right
[S39][Trial 3] --> Left
[S39][Trial 4] --> Left
[S39][Trial 5] --> Left
[S39][Trial 6] --> Left
[S39][Trial 7] --> Left
[S39][Trial 8] --> Right
[S39][Trial 9] --> Right
[S39][Trial 10] --> Left
[S40][Tri

# SSVEP

In [6]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

# -------- CONFIG --------
test_dir = r"C:\Users\zezom\PycharmProjects\HorusEye\Data\Test\SSVEP"
channels = ['C3_ref', 'CZ_ref', 'C4_ref', 'OZ_ref']
samples_per_trial = 1750
model_path = r"C:\Users\zezom\PycharmProjects\HorusEye\cnn_lstm_ssvep_final.keras"
output_csv = r"C:\Users\zezom\PycharmProjects\HorusEye\ssvep_test_predictions.csv"
ssvep_labels = ['Backward', 'Forward', 'Left', 'Right']

# Load trained model
model = tf.keras.models.load_model(model_path)
print("[INFO] Loaded SSVEP model.")

def predict_subject_trials(subject, session, data_dir, channels, samples_per_trial):
    file = f"{subject}_{session}_EEGdata_preprocessed.csv"
    path = os.path.join(data_dir, file)
    results = []
    if not os.path.exists(path):
        print(f"[WARN] File not found: {path}")
        return results

    df = pd.read_csv(path)
    n_samples = len(df)
    n_trials = n_samples // samples_per_trial

    if n_trials == 0:
        print(f"[WARN] No complete trials in {file}")
        return results

    for trial in range(1, n_trials + 1):
        start = (trial - 1) * samples_per_trial
        end = trial * samples_per_trial
        trial_data = df[channels].iloc[start:end].values.T
        # Z-score normalization per trial
        trial_data = (trial_data - trial_data.mean(axis=-1, keepdims=True)) / (trial_data.std(axis=-1, keepdims=True) + 1e-6)
        X_trial = np.expand_dims(trial_data, axis=0)
        pred_prob = model.predict(X_trial, verbose=0)
        pred_idx = np.argmax(pred_prob, axis=1)[0]
        pred_label = ssvep_labels[pred_idx]
        results.append({
            "subject": subject,
            "session": session,
            "trial": trial,
            "prediction": pred_label
        })
        print(f"[{subject}][Trial {trial}] --> {pred_label}")

    # If fewer than 10 trials, mark missing for the rest
    for trial in range(n_trials + 1, 11):
        results.append({
            "subject": subject,
            "session": session,
            "trial": trial,
            "prediction": "Missing"
        })
        print(f"[{subject}][Trial {trial}] --> Missing")

    return results

# ----- Main prediction loop -----
all_results = []
for subject in [f"S{i}" for i in range(36, 41)]:
    session = 1
    subject_results = predict_subject_trials(subject, session, test_dir, channels, samples_per_trial)
    all_results.extend(subject_results)

# ----- Save Results -----
results_df = pd.DataFrame(all_results)
results_df.to_csv(output_csv, index=False)
print(f"\nAll predictions saved to {output_csv}")


[INFO] Loaded SSVEP model.
[WARN] No complete trials in S36_1_EEGdata_preprocessed.csv
[S37][Trial 1] --> Forward
[S37][Trial 2] --> Backward
[S37][Trial 3] --> Forward
[S37][Trial 4] --> Missing
[S37][Trial 5] --> Missing
[S37][Trial 6] --> Missing
[S37][Trial 7] --> Missing
[S37][Trial 8] --> Missing
[S37][Trial 9] --> Missing
[S37][Trial 10] --> Missing
[S38][Trial 1] --> Backward
[S38][Trial 2] --> Right
[S38][Trial 3] --> Backward
[S38][Trial 4] --> Left
[S38][Trial 5] --> Left
[S38][Trial 6] --> Left
[S38][Trial 7] --> Left
[S38][Trial 8] --> Backward
[S38][Trial 9] --> Backward
[S38][Trial 10] --> Missing
[S39][Trial 1] --> Backward
[S39][Trial 2] --> Left
[S39][Trial 3] --> Left
[S39][Trial 4] --> Left
[S39][Trial 5] --> Left
[S39][Trial 6] --> Left
[S39][Trial 7] --> Missing
[S39][Trial 8] --> Missing
[S39][Trial 9] --> Missing
[S39][Trial 10] --> Missing
[S40][Trial 1] --> Backward
[S40][Trial 2] --> Left
[S40][Trial 3] --> Right
[S40][Trial 4] --> Backward
[S40][Trial 5] -->

In [7]:
import pandas as pd
path = r"C:\Users\zezom\PycharmProjects\HorusEye\Data\Test\SSVEP\S36_1_EEGdata_preprocessed.csv"
df = pd.read_csv(path)
print(len(df))  # Should be >= 1750 to get at least one trial. Otherwise, you get "No complete trials"


121
